In [3]:
import os
import sys
import importlib
import numpy as np
import pandas as pd

sys.path.append(os.path.abspath('../src'))

import utils
import preprocess

importlib.reload(utils)
importlib.reload(preprocess)

from utils import fetch_train_data
from preprocess import Preprocessor

train_df = fetch_train_data()
train_df = Preprocessor().cleanse(train_df, is_train=True)

In [2]:
# from pandas_profiling import ProfileReport

# rtr_df = pd.read_json('../data/renttherunway_data.json.gz',
#                       lines=True,
#                       compression='gzip')

# # profile = ProfileReport(rtr_df, minimal=True)
# # profile.to_notebook_iframe()

## Item info

In [4]:
# print(
#     rtr_df.groupby('item_id')['size'].agg(['unique', 'nunique',
#                                            'count']).sample(10).to_string())


In [7]:
item_df = train_df[['item_name1', 'item_name2', 'size', 'price']].copy()
item_df = item_df.astype(str, copy=False)

item_df.groupby(['item_name1', 'item_name2']).agg({
    'size': ['unique', 'nunique'],
    'price': ['unique', 'count']
}).sort_values(('price', 'count'), ascending=False).head(10)


size  \
                                                                                  unique   
item_name1     item_name2                                                                  
Cynthia Rowley Ruffle Tie Top                      [16, <NA>, 8, 14, 2, 4, 10, 6, 0, 12]   
A.L.C.         Helena Sweater                                    [<NA>, L, S, XS, M, XL]   
STYLESTALKER   Rachelle Knit Top                                     [XS, S, M, <NA>, L]   
ELLIATT        Arcadia Jumpsuit                             [XSR, SR, MR, XLR, <NA>, LR]   
Nicholas       Green Button Front Blouse          [16, <NA>, 10, 2, 12, 14, 4, 18, 6, 0]   
Trina Turk     Pink Oceanside Jumpsuit                    [8R, 6R, 10R, 12R, 2R, 4R, 0R]   
Hutch          Cropped Zia Jumpsuit          [3XR, MR, XSR, XLR, LR, 2XR, SR, 1XR, <NA>]   
Fifteen Twenty Blue Blanket Shirt Jacket                               [M, XS, S, L, XL]   
MINKPINK       All My Friends Frill Sweater                      [S, XS, M, L, <NA>, XL]   
Great Jones    Tropical Floral Romper                       [XXL, M, L, S, XL, XS, <NA>]   

                                                       price        
                                            nunique   unique count  
item_name1     item_name2                                           
Cynthia Rowley Ruffle Tie Top                     9  [265.0]    40  
A.L.C.         Helena Sweater                     5  [365.0]    40  
STYLESTALKER   Rachelle Knit Top                  4  [119.0]    40  
ELLIATT        Arcadia Jumpsuit                   5  [184.0]    40  
Nicholas       Green Button Front Blouse          9  [378.0]    40  
Trina Turk     Pink Oceanside Jumpsuit            7  [348.0]    40  
Hutch          Cropped Zia Jumpsuit               8  [270.0]    40  
Fifteen Twenty Blue Blanket Shirt Jacket          5  [264.0]    40  
MINKPINK       All My Friends Frill Sweater       5   [79.0]    40  
Great Jones    Tropical Floral Romper             6  [215.0]    40

In [8]:
size_df = item_df.groupby('item_name2')['size'].agg(['unique','nunique', 'count'])
print(size_df.sample(10).to_string())


                                                      unique  nunique  count
item_name2                                                                  
Plaid Half Zip Sweater         [L, S, XS, XXL, XXS, <NA>, M]        6     25
Cropped Faux Fur Coat      [XL, XS, XXS, M, S, <NA>, L, XXL]        7     33
Aitana Velour Crop Hoodie                  [S, M, L, XL, XS]        5     11
Flim Sweater                                    [2, 1, <NA>]        2     31
Cutout Halter Jumpsuit      [10R, 2R, 4R, <NA>, 6R, 12R, 8R]        6     19
Argyle Gem Cardigan                            [L, XS, S, M]        4      7
Jude Cold Shoulder Top                               [XS, S]        2      3
Undercover Knitted Poncho                   [M-L, P-S, <NA>]        2     33
Magenta Coat                       [S, XXL, M, XS, XL, <NA>]        5     25
60s Sweater Vest                                         [S]        1      2


In [9]:
item_df['category'] = item_df['item_name2'].str.extract(r'\b(\w+)$')

In [14]:
item_df['category'].value_counts().index

Index(['Sweater', 'Top', 'Jumpsuit', 'Jacket', 'Coat', 'Skirt', 'Romper',
       'Pants', 'Cardigan', 'Blazer', 'Blouse', 'Pullover', 'Turtleneck',
       'Vest', 'Shirt', 'Sweatshirt', 'Bomber', 'Jeans', 'Puffer', 'Trousers',
       'Joggers', 'Cape', 'Hoodie', 'Tank', 'Shorts', 'Leggings', 'Poncho',
       'Overcoat', 'Cami', 'Pant', 'Duster', 'Down', 'Tunic', 'Culottes',
       'Bralette', 'Parka', 'Fleece', 'Tee', 'Polo', 'Sweatpants', 'Zip',
       'Overalls', 'Knit', 'Up', 'Topcoat', 'Skort', 'Peacoat', 'Playsuit',
       'Crewneck', 'Dress', 'Shacket', 'Robe', 'Cardi', 'Bustier', 'Suit',
       'Bra', 'Overshirt', 'Topper', 'Corset', 'Shrug', 'Bandeau', 'Sleeve',
       'Short', 'Henley', 'Halter', 'Crew', 'Bolero', 'Skivvy', 'Bib',
       'Capris', 'Coveralls', 'Catsuit', 'Jumper', 'Kimono', 'II'],
      dtype='object')

In [11]:
item_df[item_df['category'] == 'II']


,item_name1,item_name2,size,price,category
59292,Perfect Moment,Ski Sweater II,M,320.0,II


## User info

In [35]:
user_df = train_df[[
    'user_name', 'age', 'height', 'weight', 'body_type', 'bust_size',
    'cup_size'
]].copy()
user_df.groupby('user_name').count().sort_values('age',
                                                 ascending=False).head(10)


,age,height,weight,body_type,bust_size,cup_size
user_name,,,,,,
RTR Customer,3104,2126,2291,1793,2381,2381
Lauren,1143,751,949,841,1028,1028
Sarah,1128,727,931,808,991,991
Emily,1083,707,916,782,928,928
Jennifer,1023,693,880,774,895,895
Jessica,878,575,707,635,766,766
Katie,824,520,664,603,716,716
Rachel,814,501,652,578,655,655
Amanda,790,497,657,606,711,711


In [42]:
user_df[user_df.user_name == 'Elizabeth'].sort_values(['age', 'body_type'],
                                                      ascending=False)


,user_name,age,height,weight,body_type,bust_size,cup_size
2669,Elizabeth,67.0,160.02,58.967008,PETITE,36.0,C
38007,Elizabeth,67.0,160.02,58.967008,PETITE,36.0,C
50774,Elizabeth,67.0,NaN,58.967008,PETITE,36.0,C
59804,Elizabeth,67.0,160.02,58.967008,PETITE,36.0,C
64869,Elizabeth,67.0,NaN,58.967008,PETITE,36.0,C
...,...,...,...,...,...,...,...
64503,Elizabeth,21.0,NaN,60.327785,HOURGLASS,32.0,D
67887,Elizabeth,21.0,170.18,60.327785,HOURGLASS,32.0,D
33046,Elizabeth,20.0,170.18,65.770894,NaN,NaN,NaN
26150,Elizabeth,NaN,NaN,NaN,NaN,NaN,NaN
